In [17]:
import numpy as np
from typing import Iterable, NamedTuple

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [315]:
Vector = NamedTuple('Vector', [('x', np.float32), ('y', np.float32), ('z', np.float32)])

def to_vec(v: Vector) -> np.array:
    return np.array(v).reshape((3, 1))

def normalize(v: Vector) -> np.array:
    v = np.array(v).reshape((3, 1)) 
    return v / np.linalg.norm(v, 2, 0)

Let's define a `Pose` base class holding both the position and orientation of an object.

In [331]:
class Pose:
    def __init__(self, pos: Vector):
        self.pos = to_vec(pos)  # x,y,z
        self.rot = np.matrix([[1, 0, 0], 
                              [0, 1, 0], 
                              [0, 0, 1]], dtype=np.float32)
        
    def lookat(self, pos: Vector, up: Vector=(0, 0, 1)):
        v = normalize(to_vec(pos) - self.pos)
        right = normalize(np.cross(normalize(up), v, axis=0))       
        up = normalize(np.cross(v, right, axis=0))
        self.rot = np.matrix(np.hstack([v, right, up]))
        

p = Pose((0, 0, 1))
p.lookat((5, 0, 0))
print(p.rot)

[[ 0.98058068 -0.          0.19611614]
 [ 0.          1.          0.        ]
 [-0.19611614  0.          0.98058068]]


Next, we define a sensor:

In [226]:
class Sensor(Pose):
    def __init__(self, pos: Vector=(0, 0, 0)):
        super().__init__(pos)

In [227]:
s = Sensor((1, 0, 0))
(s.rot * np.matrix([[np.cos(np.deg2rad(90)), -np.sin(np.deg2rad(90)), 0],
                    [np.sin(np.deg2rad(90)),  np.cos(np.deg2rad(90)), 0], 
                    [0, 0, 1]])) * s.pos

matrix([[6.123234e-17],
        [1.000000e+00],
        [0.000000e+00]])

And finally a Lighthouse base station:

In [361]:
class Lighthouse(Pose):
    def __init__(self, pos: Vector=(0, 0, 1)):
        super().__init__(pos)
    
    def sweep(self, sensors: Iterable[Sensor]):
        for s in sensors:
            self.get_angle(s)
            
    def get_angle(self, pose: Pose) -> np.float32:
        # TODO: Move into lighthouse reference coordinate system
        print(self.rot.T * (pose.pos - self.pos))


l = Lighthouse((0, 0, 5))
l.lookat((5, 0, 0))
l.rot

matrix([[ 0.70710678, -0.        ,  0.70710678],
        [ 0.        ,  1.        ,  0.        ],
        [-0.70710678,  0.        ,  0.70710678]])

In [362]:
l.sweep([Sensor((5, 0, 0))])

[[ 7.07106781e+00]
 [ 0.00000000e+00]
 [-1.11022302e-16]]
